# Timing the SXDS run

In this notebook we will investigate timing and requirements for the SXDS prototype run in order to estimate total computing requirements

we wil work with the file generated on iris by the following command:

```
gstatement -p IRIS-IP005-CPU -u ir-shir1 -s "2020-07-04-00:00:00" -e "2020-10-21-00:00:00" > jobs.lis
```

In [36]:
from astropy.table import Table
import numpy as np

In [37]:
t = Table.read('jobs_20201030.lis', format='ascii') #, data_start=2, delimiter=' ')

In [38]:
"I have submitted a total of {} jobs consuming a total of {} cpuhours".format(len(t), np.sum(t['CompHrs']))

'I have submitted a total of 25682 jobs consuming a total of 78615.99999999999 cpuhours'

In [39]:
t[:5]

JobID,User,Account,JobName,Partition,End,ExitCode,State,CompHrs
str12,str8,str10,str10,str10,str19,str5,str10,float64
25933230,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-13T16:18:36,127:0,FAILED,0.1
25940256,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-14T00:08:41,1:0,FAILED,74.2
25948528,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-14T14:34:59,0:0,COMPLETED,254.3
26005193,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-16T03:59:49,0:0,TIMEOUT,640.0
26027949,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-17T10:46:53,0:0,TIMEOUT,1536.4


In [40]:
t[-5:]

JobID,User,Account,JobName,Partition,End,ExitCode,State,CompHrs
str12,str8,str10,str10,str10,str19,str5,str10,float64
30479998_347,ir-shir1,iris-ip00+,phot_arra+,skylake,Unknown,0:0,RUNNING,21.1
30479998_348,ir-shir1,iris-ip00+,phot_arra+,skylake,2020-10-29T18:29:24,0:0,COMPLETED,1.1
30479998_349,ir-shir1,iris-ip00+,phot_arra+,skylake,2020-10-29T17:33:02,1:0,FAILED,0.1
30479998_350,ir-shir1,iris-ip00+,phot_arra+,skylake,2020-10-29T17:31:05,1:0,FAILED,0.1
30479998_351,ir-shir1,iris-ip00+,phot_arra+,skylake,2020-10-29T17:26:13,127:0,FAILED,0.0


In [46]:
def nameToJobType(name):
    """Take the name and return the tipe of pipetask"""
    job_type = 'UNKNOWN'
    if name.startswith('process'):
        job_type = 'processCcd'
    if name.startswith('coadd'):
        job_type = 'coadd'
    if name.startswith('phot'):
        job_type = 'photoPipe'
    return job_type
t['job_type']  = [nameToJobType(n) for n in t['JobName']]

In [47]:
t[:5]

JobID,User,Account,JobName,Partition,End,ExitCode,State,CompHrs,job_type
str12,str8,str10,str10,str10,str19,str5,str10,float64,str10
25933230,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-13T16:18:36,127:0,FAILED,0.1,UNKNOWN
25940256,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-14T00:08:41,1:0,FAILED,74.2,UNKNOWN
25948528,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-14T14:34:59,0:0,COMPLETED,254.3,UNKNOWN
26005193,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-16T03:59:49,0:0,TIMEOUT,640.0,UNKNOWN
26027949,ir-shir1,iris-ip00+,cpujob,skylake,2020-07-17T10:46:53,0:0,TIMEOUT,1536.4,UNKNOWN


In [48]:
print("""processCcd jobs run on stack images in SXDS

Total number of jobs: {}
Jobs completed: {}
mean per job: {} cpu hours
mean per completed job: {} cpu hours
Total time: {} cpu hours
Total time on completed jobs: {} cpu hours
""".format(
np.sum(t['job_type'] == 'processCcd'),
np.sum((t['job_type'] == 'processCcd') &(t['State'] == 'COMPLETED') ),
np.mean(t['CompHrs'][(t['job_type'] == 'processCcd') ]),
np.mean(t['CompHrs'][(t['job_type'] == 'processCcd') &(t['State'] == 'COMPLETED') ]),
np.sum(t['CompHrs'][(t['job_type'] == 'processCcd') ]),
np.sum(t['CompHrs'][(t['job_type'] == 'processCcd') &(t['State'] == 'COMPLETED') ])
)
)


processCcd jobs run on stack images in SXDS

Total number of jobs: 19109
Jobs completed: 12993
mean per job: 2.3228531058663453 cpu hours
mean per completed job: 3.059601323789733 cpu hours
Total time: 44387.399999999994 cpu hours
Total time on completed jobs: 39753.4 cpu hours



In [49]:
print("""coadd jobs run on stack images in SXDS

Total number of jobs: {}
Jobs completed: {}
mean per job: {} cpu hours
mean per completed job: {} cpu hours
Total time: {} cpu hours
Total time on completed jobs: {} cpu hours
""".format(
np.sum(t['job_type'] == 'coadd'),
np.sum((t['job_type'] == 'coadd') &(t['State'] == 'COMPLETED') ),
np.mean(t['CompHrs'][(t['job_type'] == 'coadd') ]),
np.mean(t['CompHrs'][(t['job_type'] == 'coadd') &(t['State'] == 'COMPLETED') ]),
np.sum(t['CompHrs'][(t['job_type'] == 'coadd') ]),
np.sum(t['CompHrs'][(t['job_type'] == 'coadd') &(t['State'] == 'COMPLETED') ])
)
)

coadd jobs run on stack images in SXDS

Total number of jobs: 353
Jobs completed: 170
mean per job: 15.865439093484417 cpu hours
mean per completed job: 32.858823529411765 cpu hours
Total time: 5600.499999999999 cpu hours
Total time on completed jobs: 5586.0 cpu hours



In [50]:
#Most of these are failures from memory shortages at teh coadd stage before I separated them
print("""photoPipe jobs run on stack images in SXDS

Total number of jobs: {}
Jobs completed: {}
mean per job: {} cpu hours
mean per completed job: {} cpu hours
Total time: {} cpu hours
Total time on completed jobs: {} cpu hours
""".format(
np.sum(t['job_type'] == 'photoPipe'),
np.sum((t['job_type'] == 'photoPipe') &(t['State'] == 'COMPLETED') ),
np.mean(t['CompHrs'][(t['job_type'] == 'photoPipe') ]),
np.mean(t['CompHrs'][(t['job_type'] == 'photoPipe') &(t['State'] == 'COMPLETED') ]),
np.sum(t['CompHrs'][(t['job_type'] == 'photoPipe') ]),
np.sum(t['CompHrs'][(t['job_type'] == 'photoPipe') &(t['State'] == 'COMPLETED') ])
)
)

photoPipe jobs run on stack images in SXDS

Total number of jobs: 352
Jobs completed: 55
mean per job: 8.338068181818182 cpu hours
mean per completed job: 11.976363636363633 cpu hours
Total time: 2935.0 cpu hours
Total time on completed jobs: 658.6999999999998 cpu hours



In [55]:
t[(t['job_type']=='coadd') & (t['State'] == 'FAILED')][:5]

JobID,User,Account,JobName,Partition,End,ExitCode,State,CompHrs,job_type
str12,str8,str10,str10,str10,str19,str5,str10,float64,str10
30458171_0,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:01,2:0,FAILED,0.1,coadd
30458171_1,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:50:49,1:0,FAILED,0.1,coadd
30458171_2,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:02,2:0,FAILED,0.1,coadd
30458171_3,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:02,2:0,FAILED,0.1,coadd
30458171_4,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:00,2:0,FAILED,0.1,coadd


In [56]:
np.unique(t[(t['job_type']=='coadd') & (t['State'] == 'FAILED')]['ExitCode'])

127:0
1:0
2:0


In [53]:
t[(t['job_type']=='coadd') ][:5]

JobID,User,Account,JobName,Partition,End,ExitCode,State,CompHrs,job_type
str12,str8,str10,str10,str10,str19,str5,str10,float64,str10
30127250,ir-shir1,iris-ip00+,coadd_852+,skylake-h+,2020-10-19T20:10:53,0:0,COMPLETED,42.0,coadd
30458171_0,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:01,2:0,FAILED,0.1,coadd
30458171_1,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:50:49,1:0,FAILED,0.1,coadd
30458171_2,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:02,2:0,FAILED,0.1,coadd
30458171_3,ir-shir1,iris-ip00+,coadd_arr+,skylake-h+,2020-10-28T14:51:02,2:0,FAILED,0.1,coadd


# 2 Calculate total times

Lets calculate some broad estimates for the main runs we will go on to perform

## 2.1 SXDS VIDEO run

This is the run used for the timing tests.

In [ ]:
n_video_sxds_images = 5263 # FRom ./1_SLurm_factory.ipynb
n_patches = 300 #not exact

print("""
Full processing of input images: {}
Full coadd of all patches: {}
Full photometry pipeline on all patches: {}
Total time for {} images and {} patches: 
""".format(
    n_video_sxds_images * np.mean(t['CompHrs'][(t['job_type'] == 'coadd') &(t['State'] == 'COMPLETED') ]),
    

## 2.2 VHS XMM run

Run over full overlap of VHS and HSC PDR2 XMM field

## 2.3 VHS complete run

A first run might not enclude combination with GRIZY data prior to LSST but we can can simply use the SXDS times to estimate here.